In [1]:

from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import tensorflow

import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
class LeNet():
    supported_dims = {28, 32}

    def __init__(self, state):
        if state.dropout:
            raise ValueError("LeNet doesn't support dropout")
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(state.nc, 6, 5, padding=2 if state.input_size == 28 else 0)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1 if state.num_classes <= 2 else state.num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x), inplace=True)
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out), inplace=True)
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out), inplace=True)
        out = F.relu(self.fc2(out), inplace=True)
        out = self.fc3(out)
        return out

In [3]:
with open('image_data.pkl','rb') as f: data = pickle.load(f)

In [4]:
d = []
for i in data:
    d.append({
        "round": i[0],
        "label": i[1],
        "img": i[2]
             })

In [5]:
import pandas as pd
df = pd.DataFrame(d)

In [6]:
df

,round,label,img
0,0,0,"[[[-0.7581059], [0.063398585], [0.3590322], [-..."
1,0,1,"[[[0.840746], [0.6316065], [-1.2231346], [-0.9..."
2,0,2,"[[[-1.7556419], [2.9030576], [0.96203077], [0...."
3,0,3,"[[[-1.3797349], [0.24783106], [-1.3950331], [-..."
4,0,4,"[[[-0.40310335], [1.279251], [0.42541805], [-0..."
...,...,...,...
295,29,5,"[[[0.97014874], [-0.62902826], [-0.94049585], ..."
296,29,6,"[[[-1.7039658], [0.7368676], [-1.3237106], [-0..."
297,29,7,"[[[0.5628856], [-2.053446], [-0.33936182], [0...."
298,29,8,"[[[-0.6217524], [-2.370984], [0.662596], [0.75..."


In [7]:
osi = df[df["round"] == 29] #one shot images

In [8]:
osi

,round,label,img
290,29,0,"[[[-1.1906161], [-1.607573], [1.5480994], [-1...."
291,29,1,"[[[0.43251985], [0.5482655], [-1.0812683], [-0..."
292,29,2,"[[[-0.45173714], [-1.7306507], [-1.1495376], [..."
293,29,3,"[[[-1.1158541], [-0.5300463], [-0.7418438], [1..."
294,29,4,"[[[-0.2796815], [0.19280227], [-0.01091018], [..."
295,29,5,"[[[0.97014874], [-0.62902826], [-0.94049585], ..."
296,29,6,"[[[-1.7039658], [0.7368676], [-1.3237106], [-0..."
297,29,7,"[[[0.5628856], [-2.053446], [-0.33936182], [0...."
298,29,8,"[[[-0.6217524], [-2.370984], [0.662596], [0.75..."
299,29,9,"[[[-1.14523], [1.3418516], [-1.1072803], [-0.2..."


In [30]:
# osi['img'].iloc[0]

In [35]:
from PIL import Image
from matplotlib import cm
img = np.uint8(osi['img'].iloc[0])
img = img[..., 0]
img = Image.fromarray(img)

img.save('test.png')

In [34]:
img.show()

In [8]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

y_train = list(map(int, list(osi["label"])))
# Scale images to the [0, 1] range
x_train = list(map(lambda x: x.astype("float32") / 255, list(osi["img"])))
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.array(x_train)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10, 28, 28, 1)
10 train samples
10000 test samples


In [9]:
model = Sequential()
model.add(keras.Input(shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1), kernel_initializer='glorot_normal'))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 48)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1200)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               307456    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                2

In [ ]:
batch_size = 3
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 11.107708930969238
Test accuracy: 0.2791000008583069
